<img align="left" src="https://lever-client-logos.s3.amazonaws.com/864372b1-534c-480e-acd5-9711f850815c-1524247202159.png" width=200>
<br></br>
<br></br>

# Train Practice

## *Data Science Unit 4 Sprint 2 Assignment 3*

Continue to use TensorFlow Keras & a sample of the [Quickdraw dataset](https://github.com/googlecreativelab/quickdraw-dataset) to build a sketch classification model. The dataset has been sampled to only 10 classes and 10000 observations per class. Using your baseline model from yesterday, hyperparameter tune it and report on your highest validation accuracy. Your singular goal today is to achieve the highest accuracy possible.

*Don't forgot to switch to GPU on Colab!*

### Hyperparameters to Tune

At a minimum, tune each of these hyperparameters using any strategy we discussed during lecture today: 
- Optimizer
- Learning Rate
- Activiation Function
  - At least 1 subparameter within the Relu activation function
- Number of Neurons in Hidden Layers
- Number of Hidden Layers
- Weight Initialization

In [ ]:
%load_ext tensorboard

import tensorflow as tf
import numpy as np
import seaborn as sns
import pandas as pd

from tensorboard.plugins.hparams import api as hp

import os
import datetime

from sklearn.model_selection import train_test_split

from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Flatten, Input
from tensorflow.keras import Sequential
from tensorflow.keras.layers import Dense
from tensorflow.keras import optimizers

The tensorboard extension is already loaded. To reload it, use:
  %reload_ext tensorboard


In [ ]:
def load_quickdraw10(path):
    data = np.load(path)
    X = data['arr_0']
    y = data['arr_1']
    X_norm = X.astype('float32')/255
    y_norm = y.astype('float32')/255
    X_train, X_test, y_train, y_test = train_test_split(
        X_norm,
        y,
        test_size=0.20,
        shuffle=True,
        random_state=42)
    
    return X_train, y_train, X_test, y_test


def train_test_model(hparams):
  model = Sequential([
    Dense(hparams[HP_NUM_UNITS], activation='relu', input_dim=784),
    Dense(10, activation='softmax')
  ])
  opt_name = hparams[HP_OPTIMIZER]
  lr = hparams[HP_LEARNING_RATE]

  if opt_name == 'sgd':
    opt = optimizers.SGD(learning_rate=lr)
  elif opt_name == 'adam':
    opt = optimizers.Adam(learning_rate=lr)
  else:
    raise Exception("Unrecognized optimizer. Must be either 'sgd' or 'adam'")

  model.compile(optimizer=opt,
                loss='sparse_categorical_crossentropy',
                metrics=['accuracy'])
  model.fit(X_train, y_train, 
            epochs=5)

  _, val_acc = model.evaluate(X_test, y_test)

  return val_acc


def run(run_dir, hparams):
  with tf.summary.create_file_writer(run_dir).as_default():
    hp.hparams(hparams)  # record the values used in this trial
    accuracy = train_test_model(hparams)
    tf.summary.scalar(METRIC_ACCURACY, accuracy, step=1)


def create_model(
    lr=0.01,
    opt=optimizers.SGD,
    act_func='relu',
    output_act_func='softmax',
    input_dim=784,
    neurons_per_layer=(32, 32, 10)):

    model = Sequential([
        Dense(neurons_per_layer[0], activation=act_func, input_dim=input_dim),
        Dense(neurons_per_layer[1], activation=act_func),
        Dense(neurons_per_layer[2], activation=output_act_func)
    ])

    model.compile(
        optimizer=opt(learning_rate=lr),
        loss='sparse_categorical_crossentropy',
        metrics=['accuracy']
    )

    return model

In [ ]:
data_path = r'./quickdraw10.npz'

X_train, y_train, X_test, y_test = load_quickdraw10(data_path)

HP_NUM_UNITS = hp.HParam('num_units', hp.Discrete([32,64]))
HP_LEARNING_RATE = hp.HParam('learning_rate', hp.RealInterval(0.001,.01))
HP_OPTIMIZER = hp.HParam('optimizer', hp.Discrete(['adam', 'sgd']))

METRIC_ACCURACY = 'accuracy'

with tf.summary.create_file_writer('logs/hparam_tuning').as_default():
  hp.hparams_config(
      hparams=[HP_NUM_UNITS, HP_LEARNING_RATE, HP_OPTIMIZER],
      metrics=[hp.Metric(METRIC_ACCURACY, display_name='Accuracy')]
  )

In [ ]:
%%time
session_num = 0 

for num_units in HP_NUM_UNITS.domain.values:
  for learning_rate in tf.linspace(HP_LEARNING_RATE.domain.min_value, HP_LEARNING_RATE.domain.max_value, num=3):
    for optimizer in HP_OPTIMIZER.domain.values:
      hparams = {
          HP_NUM_UNITS: num_units,
          HP_LEARNING_RATE:  float(learning_rate),
          HP_OPTIMIZER: optimizer
      }

      run_name = f"run-{session_num}"
      print(f"--- Starting trial: {run_name}")
      print({h.name: hparams[h] for h in hparams})
      run('logs/hparam_tuning/' + run_name, hparams)
      session_num += 1

In [ ]:
%tensorboard --logdir logs/hparam_tuning

## **W&B hyper param tuning:**

In [ ]:
!pip install --upgrade wandb

In [ ]:
!wandb login 826a25d09072881b40c973726c383743f778ddd9

wandb: Appending key for api.wandb.ai to your netrc file: /root/.netrc


In [ ]:
wandb_group = 'dustindotpy'
wandb_project = 'quickdraw_data'

In [ ]:
import wandb
from wandb.keras import WandbCallback

wandb.init(project=wandb_project, entity=wandb_group)
wandb.config.epochs = 20
wandb.config.batch_size = 32
wandb.config.n_neurons = 64
wandb.config.lr = 0.0010000
wandb.config.n_layers = 5

# Create Model
model = Sequential()
for _ in range(wandb.config.n_layers):
  model.add(Dense(wandb.config.n_neurons, activation='relu'))
model.add(Dense(10, activation='softmax'))

# Compile Model
opt = tf.keras.optimizers.Adam(learning_rate=wandb.config.lr)
model.compile(
  optimizer='adam',
  loss='sparse_categorical_crossentropy',
  metrics=['accuracy']
)

# Fit Model
model.fit(X_train, y_train, 
          validation_data=(X_test, y_test), 
          epochs=wandb.config.epochs, 
          batch_size=wandb.config.batch_size, 
          callbacks=[WandbCallback()])

## **Keras hyper parameter tuning:**

In [ ]:
!pip install keras-tuner

In [ ]:
from tensorflow import keras
from tensorflow.keras import layers
from kerastuner.tuners import RandomSearch

"""
This model Tunes:
- Number of Neurons in the Hidden Layer
- Learning Rate in Adam

"""

def build_model(hp):
    model = keras.Sequential()
    model.add(Dense(units=hp.Int('units',
                                        min_value=32,
                                        max_value=512,
                                        step=32),
                           activation='relu'))
    model.add(Dense(10, activation='softmax'))
    model.compile(
        optimizer=keras.optimizers.Adam(
            hp.Choice('learning_rate',
                      values=[1e-2, 1e-3, 1e-4])),
        loss='sparse_categorical_crossentropy',
        metrics=['accuracy'])
    return model

In [ ]:
tuner = RandomSearch(
    build_model,
    objective='val_accuracy',
    max_trials=5,
    executions_per_trial=3,
    directory='./keras-tuner-trial',
    project_name='helloworld')

In [ ]:
tuner.search_space_summary()

In [ ]:
tuner.search(X_train, y_train,
             epochs=5,
             validation_data=(X_test, y_test))

Epoch 1/5
2500/2500 [==============================] - 9s 4ms/step - loss: 0.7195 - accuracy: 0.7846 - val_loss: 0.6731 - val_accuracy: 0.8037
Epoch 2/5
2500/2500 [==============================] - 8s 3ms/step - loss: 0.6201 - accuracy: 0.8194 - val_loss: 0.6548 - val_accuracy: 0.8116
Epoch 3/5
2500/2500 [==============================] - 8s 3ms/step - loss: 0.5874 - accuracy: 0.8305 - val_loss: 0.6284 - val_accuracy: 0.8209
Epoch 4/5
2500/2500 [==============================] - 8s 3ms/step - loss: 0.5710 - accuracy: 0.8367 - val_loss: 0.6228 - val_accuracy: 0.8256
Epoch 5/5
2500/2500 [==============================] - 8s 3ms/step - loss: 0.5491 - accuracy: 0.8421 - val_loss: 0.6473 - val_accuracy: 0.8289
Epoch 1/5
2500/2500 [==============================] - 8s 3ms/step - loss: 0.7094 - accuracy: 0.7883 - val_loss: 0.6405 - val_accuracy: 0.8023
Epoch 2/5
2500/2500 [==============================] - 8s 3ms/step - loss: 0.6188 - accuracy: 0.8185 - val_loss: 0.6403 - val_accuracy: 0.8146

Epoch 1/5
2500/2500 [==============================] - 6s 2ms/step - loss: 0.7013 - accuracy: 0.7925 - val_loss: 0.5553 - val_accuracy: 0.8360
Epoch 2/5
2500/2500 [==============================] - 6s 2ms/step - loss: 0.5123 - accuracy: 0.8491 - val_loss: 0.5151 - val_accuracy: 0.8457
Epoch 3/5
2500/2500 [==============================] - 6s 2ms/step - loss: 0.4532 - accuracy: 0.8659 - val_loss: 0.4836 - val_accuracy: 0.8554
Epoch 4/5
2500/2500 [==============================] - 6s 2ms/step - loss: 0.4123 - accuracy: 0.8785 - val_loss: 0.4754 - val_accuracy: 0.8605
Epoch 5/5
2500/2500 [==============================] - 6s 2ms/step - loss: 0.3822 - accuracy: 0.8861 - val_loss: 0.4717 - val_accuracy: 0.8612
Epoch 1/5
2500/2500 [==============================] - 6s 2ms/step - loss: 0.6999 - accuracy: 0.7929 - val_loss: 0.5517 - val_accuracy: 0.8378
Epoch 2/5
2500/2500 [==============================] - 5s 2ms/step - loss: 0.5053 - accuracy: 0.8513 - val_loss: 0.4966 - val_accuracy: 0.8534

Epoch 1/5
2500/2500 [==============================] - 7s 3ms/step - loss: 0.6564 - accuracy: 0.8064 - val_loss: 0.5065 - val_accuracy: 0.8515
Epoch 2/5
2500/2500 [==============================] - 7s 3ms/step - loss: 0.4609 - accuracy: 0.8640 - val_loss: 0.4541 - val_accuracy: 0.8651
Epoch 3/5
2500/2500 [==============================] - 7s 3ms/step - loss: 0.3939 - accuracy: 0.8818 - val_loss: 0.4392 - val_accuracy: 0.8716
Epoch 4/5
2500/2500 [==============================] - 7s 3ms/step - loss: 0.3456 - accuracy: 0.8965 - val_loss: 0.4374 - val_accuracy: 0.8740
Epoch 5/5
2500/2500 [==============================] - 7s 3ms/step - loss: 0.3076 - accuracy: 0.9082 - val_loss: 0.4346 - val_accuracy: 0.8780
Epoch 1/5
2500/2500 [==============================] - 7s 3ms/step - loss: 0.6507 - accuracy: 0.8080 - val_loss: 0.5234 - val_accuracy: 0.8417
Epoch 2/5
2500/2500 [==============================] - 7s 3ms/step - loss: 0.4619 - accuracy: 0.8638 - val_loss: 0.4790 - val_accuracy: 0.8554

Epoch 1/5
2500/2500 [==============================] - 9s 4ms/step - loss: 0.7143 - accuracy: 0.7852 - val_loss: 0.6277 - val_accuracy: 0.8147
Epoch 2/5
2500/2500 [==============================] - 9s 4ms/step - loss: 0.6168 - accuracy: 0.8203 - val_loss: 0.6578 - val_accuracy: 0.8169
Epoch 3/5
2500/2500 [==============================] - 9s 4ms/step - loss: 0.5854 - accuracy: 0.8291 - val_loss: 0.6382 - val_accuracy: 0.8214
Epoch 4/5
2500/2500 [==============================] - 9s 4ms/step - loss: 0.5661 - accuracy: 0.8371 - val_loss: 0.6492 - val_accuracy: 0.8159
Epoch 5/5
2500/2500 [==============================] - 9s 4ms/step - loss: 0.5497 - accuracy: 0.8407 - val_loss: 0.6524 - val_accuracy: 0.8201
Epoch 1/5
2500/2500 [==============================] - 9s 4ms/step - loss: 0.7155 - accuracy: 0.7883 - val_loss: 0.6459 - val_accuracy: 0.8133
Epoch 2/5
2500/2500 [==============================] - 9s 3ms/step - loss: 0.6184 - accuracy: 0.8203 - val_loss: 0.6484 - val_accuracy: 0.8152

Epoch 1/5
2500/2500 [==============================] - 11s 5ms/step - loss: 0.6115 - accuracy: 0.8188 - val_loss: 0.4722 - val_accuracy: 0.8594
Epoch 2/5
2500/2500 [==============================] - 11s 4ms/step - loss: 0.4256 - accuracy: 0.8745 - val_loss: 0.4327 - val_accuracy: 0.8727
Epoch 3/5
2500/2500 [==============================] - 11s 5ms/step - loss: 0.3483 - accuracy: 0.8956 - val_loss: 0.4223 - val_accuracy: 0.8778
Epoch 4/5
2500/2500 [==============================] - 11s 4ms/step - loss: 0.2907 - accuracy: 0.9116 - val_loss: 0.4146 - val_accuracy: 0.8828
Epoch 5/5
2500/2500 [==============================] - 11s 4ms/step - loss: 0.2398 - accuracy: 0.9276 - val_loss: 0.4405 - val_accuracy: 0.8785
Epoch 1/5
2500/2500 [==============================] - 11s 4ms/step - loss: 0.6106 - accuracy: 0.8175 - val_loss: 0.4877 - val_accuracy: 0.8543
Epoch 2/5
2500/2500 [==============================] - 11s 4ms/step - loss: 0.4233 - accuracy: 0.8747 - val_loss: 0.4391 - val_accuracy:

INFO:tensorflow:Oracle triggered exit


In [ ]:
tuner.results_summary()

### Stretch Goals
- Implement Bayesian Hyper-parameter Optimization
- Select a new dataset and apply a neural network to it.
- Use a cloud base experiment tracking framework such as weights and biases
- Research potential architecture ideas for this problem. Try Lenet-10 for example. 